import pandas as pd

df_all = pd.read_pickle('data/TextData/all_df_final.pkl')
df_test = pd.read_pickle('data/TextData/test_df_final.pkl')


In [249]:
class nptr:
    def __init__(self, c):
        self.data = c
        self.left = None
        self.right = None


def display(node):
    if (node is None):
        print("")
        return
    display(node.left)
    print(node.data, end=" ")
    display(node.right)


def newNode(c):
    n = nptr(c)
    return n


def isDecimal(value):
    try:
        float(value)
        return True
    except ValueError:
        return False


def build(s):
    stN = []
    stC = []

    p = [0] * 123
    p[ord('+')] = p[ord('-')] = 1
    p[ord('/')] = p[ord('*')] = 2
    p[ord('^')] = 3
    p[ord(')')] = 0

    for i in s.split():
        # print(i)
        if i == '(':
            stC.append(i)
        # Push the operands in node stack
        elif (i.isalpha() or i.isnumeric() or i.isdecimal() or isDecimal(i)):
            t = newNode(i)
            stN.append(t)

        elif (p[ord(i)] > 0):
            while (len(stC) != 0 and stC[-1] != '(' and ((i != '^' and p[ord(stC[-1])] >= p[ord(i)])
                                                         or (i == '^' and
                                                             p[ord(stC[-1])] > p[ord(i)]))):
                t = newNode(stC[-1])
                stC.pop()
                t1 = stN[-1]
                stN.pop()
                t2 = stN[-1]
                stN.pop()
                t.left = t2
                t.right = t1

                # Push the node to the node stack
                stN.append(t)

            # Push s[i] to char stack
            stC.append(i)

        elif (i == ')'):
            while (len(stC) != 0 and stC[-1] != '('):
                t = newNode(stC[-1])
                stC.pop()
                t1 = stN[-1]
                stN.pop()
                # if(len(stN)!=0):
                t2 = stN[-1]
                stN.pop()
                t.left = t2
                t.right = t1
                stN.append(t)
            stC.pop()
    t = stN[-1]
    return t

In [243]:
def eq_parser(eq):
    try:
        eq = eq.split('=')
        eq_lhs  = '( '+eq[0]+' )'
        eq_rhs  = '( '+eq[1]+' )'
        # print(eq_lhs, eq_rhs)
        return [build(eq_lhs), build(eq_rhs)]
    except:
        print(eq)
        return []

In [204]:
eq_parser(df['eqn'][0])

[<__main__.nptr at 0x7f9435147550>, <__main__.nptr at 0x7f9435147eb0>]

In [205]:
tree = df['eqn'].apply(eq_parser)

['2 * x ', ' ( - 1 )']
['x ', ' ( ( ( - 2 ) * ( - 4 ) ) * 3 ) * ( - 1 )']
['( 2 * x ) - ( - 1 ) ', ' ( - 3 )']
['3 - ( 2 * x ) ', ' ( - 1 ) + x']
['( 3.0 * x ) + ( 2 + x ) ', ' ( - 1 ) - ( 2.0 * x )']
['( 2.0 * x ) + ( x - 2 ) ', ' ( - 1 ) - x']
['( 2.0 * x ) + ( x - 2 ) ', ' ( - 1 ) - x']
['( 2.0 * x ) + ( x - 2 ) ', ' ( - 1 ) - x']


In [206]:
len(tree)

2877

In [207]:
df_all['tree'] = tree

In [208]:
def if_variable(str):
    return str.isalpha()

def is_constant(str):
    return str.isnumeric() or str.isdecimal() or isDecimal(str)


In [220]:
def compare_tree( root1, root2):
    if root1 is None and root2 is None:
        return True

    if root1 is None or root2 is None:
        return False
    # print(root1.data, root2.data)

    if if_variable(root1.data) and if_variable(root2.data):
        return compare_tree(root1.left, root2.left) and compare_tree(root1.right, root2.right)

    if is_constant(root1.data) and is_constant(root2.data):
        return compare_tree(root1.left, root2.left) and compare_tree(root1.right, root2.right)

    if(root1.data == root2.data ):
        return compare_tree(root1.left, root2.left) and compare_tree(root1.right, root2.right)

    return False

In [221]:
compare_tree(tree[0][0], tree[0][1])

False

In [222]:
lst = []
for i in range(len(tree)):
    if len(tree[i]) == 0:
        continue
    if i%100 == 0 :
        print(i)
    tmp = []
    for j in range(len(tree)):
        if len(tree[j]) == 0:
            continue
        if compare_tree(tree[i][0], tree[j][0]) and compare_tree(tree[i][1], tree[j][1]):
            tmp.append(j)
    lst.append(tmp)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [250]:
lst_all = df_all['common']
lst_test = df_test['common']

In [252]:
print(lst_all[0])
print(lst_test[0])

[0, 1, 462, 1877, 2377, 2627]
[0]


In [234]:
df['eqn'][1]

'x = ( 8 * 0.01 ) * 2'

In [213]:
compare_tree(tree[0][0], tree[4][0])

x *


False

In [267]:
for i in lst_all[1111]:
    print(df_all['ques'][i],df_all['eqn'][i], end='\n*****\n')

as alyssa was searching through her couch cushions , she found 1 quarters , and 2 pennies in the couch . how much money in total does alyssa have ? ( 1 * 0.25 ) + ( 1 * 0.25 ) = x
*****
on wednesday , joan spent 1 half - dollars playing pinball . the next day , she spent 2 half - dollars on pinball . what was the total amount joan spent playing pinball ? ( 1 * 0.5 ) + ( 1 * 0.5 ) = x
*****


In [264]:
for i in lst_test[1]:
    print(df_test['ques'][i],df_test['eqn'][i], end='\n*****\n')

a bee has 2 legs . how many legs do 3 bees have ? x = ( 2 / 1 )
*****
christopher strolled 2 miles at 1 miles per hour . how long did christopher stroll ? x = ( 2 / 1 )
*****
if benjamin strolled 2 miles at 1 miles per hour , how long was benjamin travelling ? x = ( 2 / 1 )
*****
if lawrence walked 2 kilometers at 1 kilometers per hour , how long was lawrence walking ? x = ( 2 / 1 )
*****
mr . guzman bought 1 doughnuts packed equally into 2 boxes . how many doughnuts were in each box ? x = ( 1 / 2 )
*****
jack is inviting 1 friends to a party . he has 1 cookies . how many cookies will each friend get ? x = ( 2 / 1 )
*****
virginia has 2 eggs and 1 skittles . if she shares the eggs among 3 friends , how many eggs does each friend get ? x = ( 3 / 2 )
*****
there are 1 bottle caps in beverly's bottle cap collection . if the bottle caps are organized into 2 groups , how big is each group ? x = ( 1 / 2 )
*****
there are 1 students in the class and 2 tickets . if the tickets are divided equa

In [260]:
display(build(df_all['eqn'][0].split('=')[1]))


2 


0    [<__main__.nptr object at 0x7f9434e8f970>, <__main__.nptr object at 0x7f94337e0f40>]
1    [<__main__.nptr object at 0x7f94337e0400>, <__main__.nptr object at 0x7f94337e0910>]
2    [<__main__.nptr object at 0x7f94337b7df0>, <__main__.nptr object at 0x7f94337b7fd0>]
3    [<__main__.nptr object at 0x7f94337192b0>, <__main__.nptr object at 0x7f94337191f0>]
4    [<__main__.nptr object at 0x7f9433719b50>, <__main__.nptr object at 0x7f9433719f70>]
Name: tree, dtype: object